In [2]:
!pip install pycrypto
# !pip install crypto
!apt-get install libgmp-dev libmpfr-dev libmpc-dev
!pip install git+https://github.com/uber/ludwig.git@65ca8e762e109ea1cedbc1f666a4a8e9b4781ea1
!sudo pip3 install gmpy
!apt-get -f install
!pip2 install gmpy

  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp36-cp36m-linux_x86_64.whl size=499939 sha256=fcc32ddcb535c0e781108b0df64ce3ea0e77f17b3b6c10be40d7d9dc1dc7c505
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgmpxx4ldbl
Suggested packages:
  gmp-doc libgmp10-doc libmpfr-doc
The following NEW packages will be installed:
  libgmp-dev libgmpxx4ldbl libmpc-dev libmpfr-dev
0 upgraded, 4 newly installed, 0 to remove and 21 not upgraded.
Need to get 625 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmpxx4ldbl amd64 2:6.1.2+dfsg-2 [8,964 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmp-dev amd64 2:6.1.2+dfsg-2 [316 kB]
Get:3 http://arch

In [6]:
!pip install gmpy2

     |████████████████████████████████| 286kB 2.9MB/s 
  Created wheel for gmpy2: filename=gmpy2-2.0.8-cp36-cp36m-linux_x86_64.whl size=456722 sha256=e357ad228c685dc1add816d06093a177b51635e03a01c4b61788812470c265e4
  Stored in directory: /root/.cache/pip/wheels/33/5a/45/b4f58933bc61064f49669220932d1d5bcc0b2a1f762f25ac0e
Successfully built gmpy2


In [15]:
from random import randrange
from hashlib import sha1
from gmpy2 import xmpz, to_binary, invert, powmod, is_prime
import time

def generate_p_q(L, N):
    g = N  # g >= 160
    n = (L - 1) // g
    b = (L - 1) % g
    while True:
        # generate q
        while True:
            s = xmpz(randrange(1, 2 ** (g)))
            a = sha1(to_binary(s)).hexdigest()
            zz = xmpz((s + 1) % (2 ** g))
            z = sha1(to_binary(zz)).hexdigest()
            U = int(a, 16) ^ int(z, 16)
            mask = 2 ** (N - 1) + 1
            q = U | mask
            if is_prime(q, 20):
                break
        # generate p
        i = 0  # counter
        j = 2  # offset
        while i < 4096:
            V = []
            for k in range(n + 1):
                arg = xmpz((s + j + k) % (2 ** g))
                zzv = sha1(to_binary(arg)).hexdigest()
                V.append(int(zzv, 16))
            W = 0
            for qq in range(0, n):
                W += V[qq] * 2 ** (160 * qq)
            W += (V[n] % 2 ** b) * 2 ** (160 * n)
            X = W + 2 ** (L - 1)
            c = X % (2 * q)
            p = X - c + 1  # p = X - (c - 1)
            if p >= 2 ** (L - 1):
                if is_prime(p, 10):
                    return p, q
            i += 1
            j += n + 1


def generate_g(p, q):
    while True:
        h = randrange(2, p - 1)
        exp = xmpz((p - 1) // q)
        g = powmod(h, exp, p)
        if g > 1:
            break
    return g


def generate_keys(g, p, q):
    x = randrange(2, q)  # x < q
    y = powmod(g, x, p)
    return x, y


def generate_params(L, N):
    p, q = generate_p_q(L, N)
    g = generate_g(p, q)
    return p, q, g


def sign(M, p, q, g, x):
    if not validate_params(p, q, g):
        raise Exception("Invalid params")
    while True:
        k = randrange(2, q)  # k < q
        r = powmod(g, k, p) % q
        m = int(sha1(M).hexdigest(), 16)
        try:
            s = (invert(k, q) * (m + x * r)) % q
            return r, s
        except ZeroDivisionError:
            pass


def verify(M, r, s, p, q, g, y):
    if not validate_params(p, q, g):
        raise Exception("Invalid params")
    if not validate_sign(r, s, q):
        return False
    try:
        w = invert(s, q)
    except ZeroDivisionError:
        return False
    m = int(sha1(M).hexdigest(), 16)
    u1 = (m * w) % q
    u2 = (r * w) % q
    # v = ((g ** u1 * y ** u2) % p) % q
    v = (powmod(g, u1, p) * powmod(y, u2, p)) % p % q
    if v == r:
        return True
    return False


def validate_params(p, q, g):
    if is_prime(p) and is_prime(q):
        return True
    if powmod(g, q, p) == 1 and g > 1 and (p - 1) % q:
        return True
    return False


def validate_sign(r, s, q):
    if r < 0 and r > q:
        return False
    if s < 0 and s > q:
        return False
    return True





In [19]:
if __name__ == "__main__":
    N = 160
    L = 1024
    p, q, g = generate_params(L, N)
    x, y = generate_keys(g, p, q)

    text = "this is the text"
    M = str.encode(text, "ascii")
    start_sign_time = time.time()
    r, s = sign(M, p, q, g, x)
    end_sign_time = time.time()

    start_verify_time = time.time()
    veri = verify(M, r, s, p, q, g, y)
    end_verify_time = time.time()

    if (veri):
        print('valid signature')
    t1 = (end_sign_time - start_sign_time)
    t2 = (end_verify_time - start_verify_time)
    print('time taken to sign '+ str(t1))
    print('time taken to verify '+ str(t2))
    print('total time taken '+ str(t1+t2))

    # print("parameters are " , M, r, s, p, q, g, y, x, sep='\n')

valid signature
time taken to sign 0.013528823852539062
time taken to verify 0.013317346572875977
total time taken 0.02684617042541504
